In [1]:
import contextlib
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords, wordnet
import mmh3
import numpy as np
import contextlib
import numpy as np
import string
import nltk
import json

import spacy
import lemminflect
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])
POS = ("CC", "CD", "DT", "EX", "FW", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NN", "NNP", "NNPS", "NNS", "PDT", "PRP", "PRP$", "RB", "RBR", "RBS", "RP", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WP$", "WRB")

def lemmatize(word):
    token = nlp(str(word))[0]
    lemma = token.lemma_
    inflections = {token._.inflect(pos) for pos in POS}
    return lemma, inflections

def tokenize(sentence):
    tokenized = nlp(sentence.translate(str.maketrans('', '', string.punctuation)))
    return [token.lemma_ for token in tokenized if token.lemma_ not in en_stopwords and wordnet.synsets(token.lemma_)]

nltk.download('stopwords')
nltk.download('wordnet')
en_stopwords = set(stopwords.words('english'))

# data = datasets.load_dataset("wikipedia", "20220301.en")
# data = datasets.load_dataset("bookcorpus/bookcorpus")
# data = data['train']
data = []
for i in range (1, 1731):
    with contextlib.suppress(FileNotFoundError):
        with open(f'data/fairy_tales/{i}.txt', 'rb') as f:
            data.append(f.read())

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Yourui/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Yourui/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
neighbor_frequencies = {}
total_frequencies = {}
deltas = [-4, -3, -2, -1, 1, 2, 3, 4]

In [3]:
def update_total_frequencies(word):
    try:
        total_frequencies[word] += 1
    except KeyError:
        total_frequencies[word] = 1

In [4]:
def update_total_frequencies(word):
    try:
        total_frequencies[word] += 1
    except KeyError:
        total_frequencies[word] = 1

def update_neighbor_frequencies(i, tokenized):
    if tokenized[i] not in neighbor_frequencies.keys():
        neighbor_frequencies[tokenized[i]] = {}
    for delta in deltas:
        with contextlib.suppress(IndexError):
            try:
                neighbor_frequencies[tokenized[i]][tokenized[i+delta]] += 1
            except KeyError:
                neighbor_frequencies[tokenized[i]][tokenized[i+delta]] = 1

In [5]:
for i, row in enumerate(data):
    try:
        sentences = sent_tokenize(row.decode('cp1252').lower())
    except UnicodeDecodeError:
        sentences = sent_tokenize(row.decode('utf8').lower())
    
    for sentence in sentences:
        tokenized = tokenize(sentence)
        for i in range(len(tokenized)):
            update_total_frequencies(tokenized[i])
            update_neighbor_frequencies(i, tokenized)

    if i % 10 == 0:
        print(f"{i}th row processed")

10th row processed
10th row processed


KeyboardInterrupt: 